Copied Notebook from [this kaggle submission](https://www.kaggle.com/aravindram11/cactus-identification-with-resnet50-99-accuracy/notebook) to see how resnet compares to my highest performance models. Because the performance was similar, I did not move forward with resnet beyond looking at the initial model. 

In [1]:
import numpy as np
import pandas as pd
from skimage import io
import matplotlib.pyplot as plt
import keras.backend as K
from keras import layers
from keras.layers import Input, Add, Dense, Dropout, MaxPooling2D, Flatten
from keras.models import Model
from keras import optimizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import layer_utils
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.callbacks import History, ModelCheckpoint, Callback
from sklearn.metrics import roc_auc_score


import os
print(os.listdir("../input"))

In [2]:
!unzip ../input/train.zip

In [3]:
image = io.imread('train/00a8c7e14298819281fe1a81434d19c4.jpg')
io.imshow(image)

In [4]:
train_df = pd.read_csv('../input/train.csv')
train_df['has_cactus'].value_counts()

In [5]:
no_cactus = train_df[train_df['has_cactus'] == 0]
no_cactus

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, vertical_flip=True)

batch_size = 128
train_df.has_cactus = train_df.has_cactus.astype(str)

train_generator = train_datagen.flow_from_dataframe(dataframe = train_df[:14001],directory='train/',x_col='id',
                                                    y_col='has_cactus',class_mode='binary',batch_size=batch_size,
                                                    target_size=(32,32))
val_generator = train_datagen.flow_from_dataframe(dataframe = train_df[14001:],directory='train/',x_col='id',
                                                    y_col='has_cactus',class_mode='binary',batch_size=batch_size,
                                                    target_size=(32,32))

In [7]:
num_classes=1

def get_model():
    
    # Get base model: ResNet 50 - don't include the last set of layers dense and FC
    base_model = ResNet50(weights='imagenet',include_top=False,input_shape=(32, 32, 3))
    
    # Freeze the layers in base model
    for layer in base_model.layers:
        layer.trainable = True
        
    # Get output from base model
    base_model_output = base_model.output
    
    # Add our layers of Dense and FC at the end
    
    # FC layer and softmax
    last_layers = Flatten()(base_model_output)
    last_layers = Dense(512,activation='relu')(last_layers)
    last_layers = Dense(num_classes,activation='sigmoid',name='fcnew')(last_layers)
    
    model = Model(inputs=base_model.input,outputs=last_layers)
    return model

In [8]:
model = get_model()
optimizer = optimizers.adam(lr=0.0001)
model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
model.summary()

In [9]:
class Loss(Callback):    
    def on_train_begin(self, logs={}):
        self.losses = []
        logs['val_auc'] = 0
            
    def on_epoch_begin(self, epoch, logs={}):
        return
    
    def on_epoch_end(self, epoch, logs={}):
        self.losses.append(logs['loss'])
        
        y_p = []
        y_v = []
        for i in range(len(val_generator)):
            x_val, y_val = val_generator[i]
            y_pred = self.model.predict(x_val)
            y_p.append(y_pred)
            y_v.append(y_val)
        y_p = np.concatenate(y_p)
        y_v = np.concatenate(y_v)
        roc_auc = roc_auc_score(y_v, y_p)
        print ('\nVal AUC for epoch{}: {}'.format(epoch, roc_auc))
        logs['val_auc']=roc_auc

In [10]:
epochs = 10

loss = Loss()
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min', period=1)
history = model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.n//batch_size,
                   validation_data=val_generator,
                   validation_steps=val_generator.n//batch_size,
                   epochs=epochs,
                   callbacks=[loss,checkpoint]
    )

In [11]:

def visualize_training_results(history):
    '''
    From https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
    
    Input: keras history object (output from trained model)
    '''
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    fig.suptitle('Model Results')

    # summarize history for accuracy
    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_ylabel('Accuracy')
    ax1.legend(['train', 'test'], loc='upper left')
    # summarize history for loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_ylabel('Loss')
    ax2.legend(['train', 'test'], loc='upper left')
    
    plt.xlabel('Epoch')
    plt.show()
    
def plot_performance(hist):
    """ Returns 4 plots comparing Training and Validation data. 
    First plot returns training and validation accuracy. 
    Second plot returns training and validation loss. 
    Third plot returns training and validation F1-Scores. 
    Fourth plot returns training and validation recall scores. 
    
    hist: input history model containing train images, labels, and validation data. t"""
    
    hist_ = hist.history
    epochs = hist.epoch
    
    plt.plot(epochs, hist_['accuracy'], label='Training Accuracy')
    plt.plot(epochs, hist_['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.figure()
    
    plt.plot(epochs, hist_['loss'], label='Training loss')
    plt.plot(epochs, hist_['val_loss'], label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    recall = np.array(hist_['recall'])
    precision = np.array(hist_['precision'])
    val_recall = np.array(hist_['val_recall'])
    val_precision = np.array(hist_['val_precision'])
    plt.figure()
    
    plt.plot(epochs, 
             2*((recall * precision)/(recall + precision)), 
             label='Training f1')
    plt.plot(epochs, 
             2*((val_recall * val_precision)/(val_recall + val_precision)), 
             label='Validation f1')
    plt.title('Training and validation F1-Score')
    plt.legend()
    plt.figure()
    
    plt.plot(epochs, recall, label = "Training Recall")
    plt.plot(epochs, val_recall, label = "Validation Recall")
    plt.title("Training and Validation Recall Scores")
    plt.legend()
    plt.figure()
    
    plt.show()